In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,  classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
df = pd.read_csv("loan_dataset_final.csv", encoding='latin1')

df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,emp_title,emp_length,home_ownership,...,collections_12_mths_ex_med,pub_rec_bankruptcies,interest_rate,revol_utilization,number_bc_gt_75,fico_score,lti,month_since_oldest_tl,race_name,gender
0,1000007,1225615,5150,5150,5150.0,60 months,132.58,atlantic tomorrows office,1 year,RENT,...,0.0,1.0,0.1864,0.873,2,709,0.147176,124.0,White,Female
1,1000030,1225638,20000,20000,20000.0,36 months,635.07,The Red Threads Inc.,6 years,RENT,...,0.0,0.0,0.0890,0.354,0,744,0.277778,149.0,African American,Female
2,1000033,1225642,12800,12800,12750.0,60 months,316.54,T-Mobile USA Inc,9 years,MORTGAGE,...,0.0,0.0,0.1677,0.754,0,719,0.156655,176.0,White,Female
3,1000045,1225655,14000,14000,14000.0,60 months,349.98,Trader Joe's,9 years,MORTGAGE,...,0.0,0.0,0.1727,0.357,0,714,0.194444,140.0,White,Female
4,1000067,1225680,15000,15000,14975.0,60 months,370.94,Truevance Engineering,< 1 year,RENT,...,0.0,0.0,0.1677,0.369,0,709,0.208333,136.0,White,Female


In [3]:
_, test_output = train_test_split(df, test_size=0.1, random_state=26)

test_output[" "] = " "

In [4]:
df = df.drop(['emp_title', 'title', 'desc', 'collections_12_mths_ex_med'], axis=1)

df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,emp_length,home_ownership,annual_inc,...,collection_recovery_fee,pub_rec_bankruptcies,interest_rate,revol_utilization,number_bc_gt_75,fico_score,lti,month_since_oldest_tl,race_name,gender
0,1000007,1225615,5150,5150,5150.0,60 months,132.58,1 year,RENT,34992.0,...,0.0,1.0,0.1864,0.873,2,709,0.147176,124.0,White,Female
1,1000030,1225638,20000,20000,20000.0,36 months,635.07,6 years,RENT,72000.0,...,0.0,0.0,0.0890,0.354,0,744,0.277778,149.0,African American,Female
2,1000033,1225642,12800,12800,12750.0,60 months,316.54,9 years,MORTGAGE,81708.0,...,0.0,0.0,0.1677,0.754,0,719,0.156655,176.0,White,Female
3,1000045,1225655,14000,14000,14000.0,60 months,349.98,9 years,MORTGAGE,72000.0,...,0.0,0.0,0.1727,0.357,0,714,0.194444,140.0,White,Female
4,1000067,1225680,15000,15000,14975.0,60 months,370.94,< 1 year,RENT,72000.0,...,0.0,0.0,0.1677,0.369,0,709,0.208333,136.0,White,Female


In [5]:
df = df.replace(['Current', 'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid', 'Charged Off', 'In Grace Period', 
                 'Late (31-120 days)', 'Late (16-30 days)', 'Does not meet the credit policy. Status:Charged Off', 'Default'],
                 [1, 1, 1, 0, 1, 0, 0, 0, 0])

df.head()

C:\Users\unnat\AppData\Local\Temp\ipykernel_106768\2976852631.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(['Current', 'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid', 'Charged Off', 'In Grace Period',


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,emp_length,home_ownership,annual_inc,...,collection_recovery_fee,pub_rec_bankruptcies,interest_rate,revol_utilization,number_bc_gt_75,fico_score,lti,month_since_oldest_tl,race_name,gender
0,1000007,1225615,5150,5150,5150.0,60 months,132.58,1 year,RENT,34992.0,...,0.0,1.0,0.1864,0.873,2,709,0.147176,124.0,White,Female
1,1000030,1225638,20000,20000,20000.0,36 months,635.07,6 years,RENT,72000.0,...,0.0,0.0,0.0890,0.354,0,744,0.277778,149.0,African American,Female
2,1000033,1225642,12800,12800,12750.0,60 months,316.54,9 years,MORTGAGE,81708.0,...,0.0,0.0,0.1677,0.754,0,719,0.156655,176.0,White,Female
3,1000045,1225655,14000,14000,14000.0,60 months,349.98,9 years,MORTGAGE,72000.0,...,0.0,0.0,0.1727,0.357,0,714,0.194444,140.0,White,Female
4,1000067,1225680,15000,15000,14975.0,60 months,370.94,< 1 year,RENT,72000.0,...,0.0,0.0,0.1677,0.369,0,709,0.208333,136.0,White,Female


In [6]:
df = df.replace(['1 year', '6 years', '9 years', '< 1 year', '8 years', '10+ years', '5 years', '2 years', '3 years', '4 years', '7 years'],
                [1, 6, 9, 0, 8, 10, 5, 2, 3, 4, 7])

df['emp_length'] = df['emp_length'].fillna(0)

df.head()

C:\Users\unnat\AppData\Local\Temp\ipykernel_106768\1927801908.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(['1 year', '6 years', '9 years', '< 1 year', '8 years', '10+ years', '5 years', '2 years', '3 years', '4 years', '7 years'],


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,emp_length,home_ownership,annual_inc,...,collection_recovery_fee,pub_rec_bankruptcies,interest_rate,revol_utilization,number_bc_gt_75,fico_score,lti,month_since_oldest_tl,race_name,gender
0,1000007,1225615,5150,5150,5150.0,60 months,132.58,1.0,RENT,34992.0,...,0.0,1.0,0.1864,0.873,2,709,0.147176,124.0,White,Female
1,1000030,1225638,20000,20000,20000.0,36 months,635.07,6.0,RENT,72000.0,...,0.0,0.0,0.0890,0.354,0,744,0.277778,149.0,African American,Female
2,1000033,1225642,12800,12800,12750.0,60 months,316.54,9.0,MORTGAGE,81708.0,...,0.0,0.0,0.1677,0.754,0,719,0.156655,176.0,White,Female
3,1000045,1225655,14000,14000,14000.0,60 months,349.98,9.0,MORTGAGE,72000.0,...,0.0,0.0,0.1727,0.357,0,714,0.194444,140.0,White,Female
4,1000067,1225680,15000,15000,14975.0,60 months,370.94,0.0,RENT,72000.0,...,0.0,0.0,0.1677,0.369,0,709,0.208333,136.0,White,Female


In [7]:
df['annual_inc'] = df['annual_inc'].fillna(df['annual_inc'].mean())

In [8]:
df['Division'] = df['loan_amnt'] / df['annual_inc']
df['lti'] = df['lti'].fillna(df['Division'])


df = df.drop(columns=['Division'])

In [9]:
df['mths_since_last_delinq'] = df['mths_since_last_delinq'].fillna(df['mths_since_last_delinq'].max()*5)
df['mths_since_last_record'] = df['mths_since_last_record'].fillna(df['mths_since_last_record'].max()*5)
df['month_since_oldest_tl'] = df['month_since_oldest_tl'].fillna(df['month_since_oldest_tl'].max()*5)

In [10]:
df['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].fillna(0)
df['delinq_2yrs'] = df['delinq_2yrs'].fillna(0)
df['inq_last_6mths'] = df['inq_last_6mths'].fillna(0)
df['open_acc'] = df['open_acc'].fillna(0)
df['pub_rec'] = df['pub_rec'].fillna(0)
df['total_acc'] = df['total_acc'].fillna(0)
# df['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].fillna(0)
df['revol_utilization'] = df['revol_utilization'].fillna(0)

In [11]:
df.isnull().sum()

id                         0
member_id                  0
loan_amnt                  0
funded_amnt                0
funded_amnt_inv            0
term                       0
installment                0
emp_length                 0
home_ownership             0
annual_inc                 0
verification_status        0
issue_d                    0
loan_status                0
pymnt_plan                 0
purpose                    0
addr_state                 0
dti                        0
delinq_2yrs                0
inq_last_6mths             0
mths_since_last_delinq     0
mths_since_last_record     0
open_acc                   0
pub_rec                    0
revol_bal                  0
total_acc                  0
out_prncp                  0
out_prncp_inv              0
total_pymnt                0
total_pymnt_inv            0
total_rec_prncp            0
total_rec_int              0
total_rec_late_fee         0
recoveries                 0
collection_recovery_fee    0
pub_rec_bankru

In [12]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
continuous_columns = df.select_dtypes(include=['int64', 'float64']).columns

categorical_columns

Index(['term', 'home_ownership', 'verification_status', 'issue_d',
       'pymnt_plan', 'purpose', 'addr_state', 'race_name', 'gender'],
      dtype='object')

In [13]:
continuous_columns = continuous_columns.to_list()

continuous_columns.remove('loan_status')
continuous_columns.remove('id')
continuous_columns.remove('member_id')

In [14]:
one_hot_encoded = pd.get_dummies(df['term'], prefix='term')

one_hot_encoded.astype(int).head()

,term_36 months,term_60 months
0,0,1
1,1,0
2,0,1
3,0,1
4,0,1


In [15]:
for col in categorical_columns:
    one_hot_encoded = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df, one_hot_encoded.astype(int)], axis=1)

    df = df.drop(columns=[col])

df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,emp_length,annual_inc,loan_status,dti,...,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,race_name_African American,race_name_Asian,race_name_Other,race_name_White,gender_Female,gender_Male
0,1000007,1225615,5150,5150,5150.0,132.58,1.0,34992.0,1,17.59,...,0,0,0,0,0,0,0,1,1,0
1,1000030,1225638,20000,20000,20000.0,635.07,6.0,72000.0,1,7.80,...,0,0,0,0,1,0,0,0,1,0
2,1000033,1225642,12800,12800,12750.0,316.54,9.0,81708.0,1,14.22,...,0,0,0,0,0,0,0,1,1,0
3,1000045,1225655,14000,14000,14000.0,349.98,9.0,72000.0,1,20.35,...,0,0,0,0,0,0,0,1,1,0
4,1000067,1225680,15000,15000,14975.0,370.94,0.0,72000.0,1,20.50,...,0,0,0,0,0,0,0,1,1,0


# Training on original dataset

In [16]:
x = df.drop(['loan_status', 'id', 'member_id'], axis=1)
y = df['loan_status']

# Standardize continuous variables
scaler = StandardScaler()
x[continuous_columns] = scaler.fit_transform(x[continuous_columns])

x.head()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,race_name_African American,race_name_Asian,race_name_Other,race_name_White,gender_Female,gender_Male
0,-0.801490,-0.793581,-0.699679,-0.909624,-1.051618,-0.532738,0.626942,-0.297409,0.990289,0.760435,...,0,0,0,0,0,0,0,1,1,0
1,1.202328,1.284264,1.382602,1.495500,0.334139,0.044677,-0.828553,-0.297409,0.405887,0.760435,...,0,0,0,0,1,0,0,0,1,0
2,0.230780,0.276824,0.366000,-0.029116,1.165594,0.196145,0.125918,1.654796,-0.178515,-1.385193,...,0,0,0,0,0,0,0,1,1,0
3,0.392705,0.444731,0.541277,0.130942,1.165594,0.044677,1.037275,-0.297409,-0.178515,0.760435,...,0,0,0,0,0,0,0,1,1,0
4,0.527642,0.584653,0.677992,0.231265,-1.328770,0.044677,1.059576,-0.297409,-0.762917,0.760435,...,0,0,0,0,0,0,0,1,1,0


In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=26)

In [18]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=101)
x_train, y_train = smote.fit_resample(x_train, y_train)

In [19]:
status_list = ['Approved' if x == 1 else 'Rejected' for x in y_test]

test_output['Actual_Status'] = status_list

Fitting Logistic Regression


In [20]:
logistic_model = LogisticRegression()

logistic_model.fit(x_train, y_train)

y_pred = logistic_model.predict(x_test)
y_pred2 = logistic_model.predict(x_train)

In [21]:
# train_acc = accuracy_score(y_train, y_pred2)
# accuracy = accuracy_score(y_test, y_pred)
# print("train Accuracy:", train_acc)
# print("Test Accuracy:", accuracy)

In [22]:
print("Classification Report for test set:")
print(classification_report(y_true=y_test, y_pred=y_pred))
print("\nClassification Report for train set:")
print(classification_report(y_true=y_train, y_pred=y_pred2))

Classification Report for test set:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       635
           1       0.99      1.00      1.00      3619

    accuracy                           1.00      4254
   macro avg       1.00      0.98      0.99      4254
weighted avg       1.00      1.00      1.00      4254


Classification Report for train set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     32484
           1       1.00      1.00      1.00     32484

    accuracy                           1.00     64968
   macro avg       1.00      1.00      1.00     64968
weighted avg       1.00      1.00      1.00     64968



In [23]:
status_list = ['Approved' if x == 1 else 'Rejected' for x in y_pred]

test_output['Logistic Regression Output'] = status_list

In [24]:
feature_importance = pd.DataFrame({'feature': x.columns, 'coefficient': logistic_model.coef_[0]})

feature_importance['abs_coefficient'] = feature_importance['coefficient'].abs()
feature_importance = feature_importance.sort_values(by='abs_coefficient', ascending=False)

In [25]:
# pd.set_option('display.max_rows', None)  # Set to None to display all rows
# pd.set_option('display.max_columns', None)  # Set to None to display all columns

# print(feature_importance)

Fitting Decision Tree on encoded dataset

In [26]:
# # Define the hyperparameter grid
# param_dist = {
#     'criterion': ['gini', 'entropy'],
#     'splitter': ['best', 'random'],
#     'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
#     'min_samples_split': np.arange(2, 11),
#     'min_samples_leaf': np.arange(1, 11),
#     'max_features': [None, 'auto', 'sqrt', 'log2']
# }

# # Initialize the Decision Tree classifier
# clf = DecisionTreeClassifier()

# # Initialize RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=clf,
#     param_distributions=param_dist,
#     n_iter=100,  # Number of parameter settings sampled
#     cv=5,  # 5-fold cross-validation
#     verbose=1,
#     random_state=42,
#     n_jobs=-1  # Use all available cores
# )

# # Fit RandomizedSearchCV
# random_search.fit(x_train, y_train)

In [27]:
# # Print the best parameters and best score
# print("Best parameters found: ", random_search.best_params_)
# print(f"Best cross-validation score: {random_search.best_score_}")

# # Evaluate on the test set
# best_clf = random_search.best_estimator_
# test_score = best_clf.score(x_test, y_test)
# print(f"Test set score: {test_score}")

Random Forest fitted on processed dataset

In [28]:
# # Step 5: Define the parameter grid
# param_dist = {
#     'n_estimators': randint(10, 200),
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
#     'min_samples_split': randint(2, 20),
#     'min_samples_leaf': randint(1, 20),
#     'bootstrap': [True, False]
# }

# # Step 6: Set up the RandomizedSearchCV
# rf_model = RandomForestClassifier(random_state=42)
# random_search = RandomizedSearchCV(
#     estimator=rf_model,
#     param_distributions=param_dist,
#     n_iter=100,  # Number of parameter settings that are sampled
#     cv=5,       # 5-fold cross-validation
#     verbose=2,
#     random_state=42,
#     n_jobs=-1   # Use all available cores
# )

# # Step 7: Fit the model
# random_search.fit(x_train, y_train)

In [29]:
# # Step 8: Evaluate the best model
# best_rf_model = random_search.best_estimator_
# y_pred = best_rf_model.predict(x_test)
# y_pred_train = best_rf_model.predict(x_train)

# print(f"Best Parameters: {random_search.best_params_}")
# print(f"train accuracy = {accuracy_score(y_train, y_pred_train)}")
# print(f"Test accuracy = {accuracy_score(y_test, y_pred)}")

Decision Tree on Non-Scaled Dataset

In [30]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,emp_length,annual_inc,loan_status,dti,...,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,race_name_African American,race_name_Asian,race_name_Other,race_name_White,gender_Female,gender_Male
0,1000007,1225615,5150,5150,5150.0,132.58,1.0,34992.0,1,17.59,...,0,0,0,0,0,0,0,1,1,0
1,1000030,1225638,20000,20000,20000.0,635.07,6.0,72000.0,1,7.80,...,0,0,0,0,1,0,0,0,1,0
2,1000033,1225642,12800,12800,12750.0,316.54,9.0,81708.0,1,14.22,...,0,0,0,0,0,0,0,1,1,0
3,1000045,1225655,14000,14000,14000.0,349.98,9.0,72000.0,1,20.35,...,0,0,0,0,0,0,0,1,1,0
4,1000067,1225680,15000,15000,14975.0,370.94,0.0,72000.0,1,20.50,...,0,0,0,0,0,0,0,1,1,0


In [31]:
label = df.drop(['id', 'member_id', 'loan_status'], axis=1)
output = df['loan_status']

x_train_or, x_test_or, y_train_or, y_test_or = train_test_split(label, output, test_size=0.1, random_state=26)

# from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=101)
(x_train_or, y_train_or) = smote.fit_resample(x_train_or, y_train_or)

# Define the hyperparameter grid
param_dist = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'min_samples_split': np.arange(2, 11),
    'min_samples_leaf': np.arange(1, 11),
    'max_features': [None, 'sqrt', 'log2']
}

# Initialize the Decision Tree classifier
clf2 = DecisionTreeClassifier()

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=clf2,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings sampled
    cv=5,  # 5-fold cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit RandomizedSearchCV
random_search.fit(x_train_or, y_train_or)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 10, 20, 30, 40, 50,
                                                      60, 70, 80, 90, 100],
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'splitter': ['best', 'random']},
                   random_state=42, verbose=1)

In [32]:
# Print the best parameters and best score
print("Best parameters found: ", random_search.best_params_)
print(f"Best cross-validation score: {random_search.best_score_}")

# Evaluate on the test set
best_clf = random_search.best_estimator_
test_score = best_clf.score(x_test_or, y_test_or)
print(f"Test set score: {test_score}")

Best parameters found:  {'splitter': 'best', 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 50, 'criterion': 'entropy'}
Best cross-validation score: 0.9937662873631972
Test set score: 0.995768688293371


In [33]:
y_pred = best_clf.predict(x_test_or)
y_pred2 = best_clf.predict(x_train_or)

print("Classification Report for test set:")
print(classification_report(y_true=y_test, y_pred=y_pred))
print("\nClassification Report for train set:")
print(classification_report(y_true=y_train, y_pred=y_pred2))

Classification Report for test set:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       635
           1       1.00      1.00      1.00      3619

    accuracy                           1.00      4254
   macro avg       0.99      0.99      0.99      4254
weighted avg       1.00      1.00      1.00      4254


Classification Report for train set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     32484
           1       1.00      1.00      1.00     32484

    accuracy                           1.00     64968
   macro avg       1.00      1.00      1.00     64968
weighted avg       1.00      1.00      1.00     64968



In [34]:
status_list = ['Approved' if x == 1 else 'Rejected' for x in y_pred]

test_output['Decision Tree Output'] = status_list

Random Forest on unscaled data

In [36]:
# Step 5: Define the parameter grid
param_dist = {
    'n_estimators': randint(10, 200),
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'bootstrap': [True, False]
}

# Step 6: Set up the RandomizedSearchCV
rf_model2 = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(
    estimator=rf_model2,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings that are sampled
    cv=5,       # 5-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Step 7: Fit the model
random_search.fit(x_train_or, y_train_or)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30, 40, 50,
                                                      60, 70, 80, 90, 100],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001ED471E26E0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001ED56AEDDE0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001ED471FF310>},
                   random_state=42, verbose=2)

In [37]:
# Step 8: Evaluate the best model
best_rf_model = random_search.best_estimator_
y_pred = best_rf_model.predict(x_test_or)
y_pred_train = best_rf_model.predict(x_train_or)

print(f"Best Parameters: {random_search.best_params_}")
print(f"train accuracy = {accuracy_score(y_train_or, y_pred_train)}")
print(f"Test accuracy = {accuracy_score(y_test_or, y_pred)}")

Best Parameters: {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 9, 'n_estimators': 83}
train accuracy = 0.999907646841522
Test accuracy = 0.9950634696755994


In [38]:
status_list = ['Approved' if x == 1 else 'Rejected' for x in y_pred]

test_output['Random Forest Output'] = status_list

print("Classification Report for test set:")
print(classification_report(y_true=y_test, y_pred=y_pred))
print("\nClassification Report for train set:")
print(classification_report(y_true=y_train, y_pred=y_pred_train))

Classification Report for test set:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       635
           1       0.99      1.00      1.00      3619

    accuracy                           1.00      4254
   macro avg       1.00      0.98      0.99      4254
weighted avg       1.00      1.00      1.00      4254


Classification Report for train set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     32484
           1       1.00      1.00      1.00     32484

    accuracy                           1.00     64968
   macro avg       1.00      1.00      1.00     64968
weighted avg       1.00      1.00      1.00     64968



XGBoost on unscaled data

In [39]:
param_dist = {
    'n_estimators': randint(50, 200),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 0.5)
}

# Step 6: Set up the RandomizedSearchCV
xgb_model = XGBClassifier(random_state=101, use_label_encoder=False, eval_metric='logloss')
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings that are sampled
    cv=5,       # 5-fold cross-validation
    verbose=2,
    random_state=101,
    n_jobs=-1   # Use all available cores
)

# Step 7: Fit the model
random_search.fit(x_train_or, y_train_or)

# Step 8: Evaluate the best model
best_xgb_model = random_search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [40]:
# print(f"Best Parameters: {random_search.best_params_}")
# print(f"train accuracy: {accuracy_score(y_train_or, best_xgb_model.predict(x_train_or))}")
# print(f"Test accuracy: {accuracy_score(y_test_or, best_xgb_model.predict(x_test_or))}")

In [41]:
status_list = ['Approved' if x == 1 else 'Rejected' for x in best_xgb_model.predict(x_test_or)]

test_output['XGBoost Output'] = status_list

print("Classification Report for test set:")
print(classification_report(y_true=y_test, y_pred=best_xgb_model.predict(x_test_or)))
print("\nClassification Report for train set:")
print(classification_report(y_true=y_train, y_pred=best_xgb_model.predict(x_train_or)))

Classification Report for test set:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       635
           1       1.00      1.00      1.00      3619

    accuracy                           1.00      4254
   macro avg       1.00      0.99      0.99      4254
weighted avg       1.00      1.00      1.00      4254


Classification Report for train set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     32484
           1       1.00      1.00      1.00     32484

    accuracy                           1.00     64968
   macro avg       1.00      1.00      1.00     64968
weighted avg       1.00      1.00      1.00     64968



In [42]:
with pd.ExcelWriter('Output.xlsx', mode='w', engine='openpyxl') as writer:
    test_output.to_excel(writer, sheet_name='Predictions on Entire Dataset', index=False)

In [43]:
# writer = pd.ExcelWriter(, engine='openpyxl')
# test_output.to_excel(writer, sheet_name=)
# writer.save()

In [44]:
# x_train_or.head()

In [45]:
# plt.figure(figsize=(200,100))
# xgb.plot_importance(best_xgb_model)


# plt.show()

In [46]:
# # Get the feature importances as a dictionary
# importances = best_xgb_model.get_booster().get_score(importance_type='weight')

# # Convert to a DataFrame for easier viewing
# import pandas as pd
# importance_df = pd.DataFrame.from_dict(importances, orient='index', columns=['importance'])
# importance_df.sort_values(by='importance', ascending=False, inplace=True)

# # Display the DataFrame
# print(importance_df)

In [47]:
# sample_index = np.random.choice(x_test_or.shape[0], 1, replace=False)
# x_test_sample = x_test_or.iloc[sample_index]

# print(sample_index)
# x_test_sample.unstack()

In [48]:
# import shap

# # # Explain the model's predictions with SHAP
# # explainer = shap.Explainer(best_xgb_model, x_train_or)
# # shap_values = explainer.shap_values(x_test_sample)  # x_test_sample should be a single data point

# # # Visualize the explanation
# # shap.initjs()
# # shap.force_plot(explainer.expected_value, shap_values, x_test_sample)

In [49]:
# # Summary plot
# shap_values2 = explainer.shap_values(x_train_or)
# shap.summary_plot(shap_values2, x_train_or)

# Training on dataset with removed sensitive fields

In [50]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,emp_length,annual_inc,loan_status,dti,...,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,race_name_African American,race_name_Asian,race_name_Other,race_name_White,gender_Female,gender_Male
0,1000007,1225615,5150,5150,5150.0,132.58,1.0,34992.0,1,17.59,...,0,0,0,0,0,0,0,1,1,0
1,1000030,1225638,20000,20000,20000.0,635.07,6.0,72000.0,1,7.80,...,0,0,0,0,1,0,0,0,1,0
2,1000033,1225642,12800,12800,12750.0,316.54,9.0,81708.0,1,14.22,...,0,0,0,0,0,0,0,1,1,0
3,1000045,1225655,14000,14000,14000.0,349.98,9.0,72000.0,1,20.35,...,0,0,0,0,0,0,0,1,1,0
4,1000067,1225680,15000,15000,14975.0,370.94,0.0,72000.0,1,20.50,...,0,0,0,0,0,0,0,1,1,0


In [51]:
# Word to filter columns by
word_to_filter = ('addr_state', 'gender', 'race_name')

# Identify columns to drop
columns_to_drop = [col for col in df.columns if col.startswith(word_to_filter)]

columns_to_drop

['addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY',
 'race_name_African American',
 'race_name_Asian',
 'race_name_Other',
 'race_name_White',
 'gender_

In [52]:
df = df.drop(columns=columns_to_drop)

df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,emp_length,annual_inc,loan_status,dti,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,1000007,1225615,5150,5150,5150.0,132.58,1.0,34992.0,1,17.59,...,0,0,0,0,0,0,0,0,0,0
1,1000030,1225638,20000,20000,20000.0,635.07,6.0,72000.0,1,7.80,...,0,0,0,0,1,0,0,0,0,0
2,1000033,1225642,12800,12800,12750.0,316.54,9.0,81708.0,1,14.22,...,1,0,0,0,0,0,0,0,0,0
3,1000045,1225655,14000,14000,14000.0,349.98,9.0,72000.0,1,20.35,...,0,0,0,0,0,0,0,0,0,0
4,1000067,1225680,15000,15000,14975.0,370.94,0.0,72000.0,1,20.50,...,0,0,0,0,0,0,0,1,0,0


In [53]:
x = df.drop(['loan_status', 'id', 'member_id'], axis=1)
y = df['loan_status']

# Standardize continuous variables
# scaler = StandardScaler()
# x[continuous_columns] = scaler.fit_transform(x[continuous_columns])

x.head()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,5150,5150,5150.0,132.58,1.0,34992.0,17.59,0.0,3.0,600.0,...,0,0,0,0,0,0,0,0,0,0
1,20000,20000,20000.0,635.07,6.0,72000.0,7.80,0.0,2.0,600.0,...,0,0,0,0,1,0,0,0,0,0
2,12800,12800,12750.0,316.54,9.0,81708.0,14.22,1.0,1.0,15.0,...,1,0,0,0,0,0,0,0,0,0
3,14000,14000,14000.0,349.98,9.0,72000.0,20.35,0.0,1.0,600.0,...,0,0,0,0,0,0,0,0,0,0
4,15000,15000,14975.0,370.94,0.0,72000.0,20.50,0.0,0.0,600.0,...,0,0,0,0,0,0,0,1,0,0


In [54]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=26)

smote = SMOTE(random_state=101)
(x_train, y_train) = smote.fit_resample(x_train, y_train)

In [55]:
# param_dist = {
#     'n_estimators': randint(50, 200),
#     'learning_rate': uniform(0.01, 0.3),
#     'max_depth': randint(3, 10),
#     'min_child_weight': randint(1, 10),
#     'subsample': uniform(0.6, 0.4),
#     'colsample_bytree': uniform(0.6, 0.4),
#     'gamma': uniform(0, 0.5)
# }

# # Step 6: Set up the RandomizedSearchCV
# xgb_model = XGBClassifier(random_state=101, use_label_encoder=False, eval_metric='logloss')
# random_search = RandomizedSearchCV(
#     estimator=xgb_model,
#     param_distributions=param_dist,
#     n_iter=100,  # Number of parameter settings that are sampled
#     cv=5,       # 5-fold cross-validation
#     verbose=2,
#     random_state=101,
#     n_jobs=-1   # Use all available cores
# )

# # Step 7: Fit the model
# random_search.fit(x_train, y_train)

# # Step 8: Evaluate the best model
# best_xgb_model = random_search.best_estimator_

In [56]:
# print(f"Best Parameters: {random_search.best_params_}")
# print(f"train accuracy: {accuracy_score(y_train, best_xgb_model.predict(x_train))}")
# print(f"Test accuracy: {accuracy_score(y_test, best_xgb_model.predict(x_test))}")

In [57]:
# # Compute the confusion matrix
# cm = confusion_matrix(y_test, y_pred)

# # Display the confusion matrix
# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot()

# # Show the plot
# plt.show()

In [58]:
# tn, fp, fn, tp = cm.ravel()

# # Calculate recall for each class
# recall_pos = tp / (tp + fn)  # Recall for the positive class
# recall_neg = tn / (tn + fp)  # Recall for the negative class

# # Calculate balanced accuracy
# balanced_accuracy = (recall_pos + recall_neg) / 2

# print(f"Balanced Accuracy: {balanced_accuracy}")

# Trying to handle class imbalance

In [59]:
x_train.head()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,15000,15000,15000.0,368.69,10.0,48000.0,15.55,0.0,1.0,600.0,...,0,0,0,0,0,0,0,0,0,0
1,10000,10000,10000.0,344.65,8.0,90305.0,9.51,0.0,0.0,55.0,...,0,0,0,0,0,0,0,0,0,0
2,11000,11000,10950.0,342.22,10.0,22000.0,15.16,0.0,2.0,600.0,...,0,0,0,0,0,0,0,0,0,0
3,1925,1925,0.0,64.79,1.0,31000.0,21.65,0.0,11.0,600.0,...,0,0,0,0,0,0,0,0,0,1
4,15000,15000,14825.0,492.06,5.0,160000.0,8.75,0.0,4.0,600.0,...,1,0,0,0,0,0,0,0,0,0


In [60]:
(y_train == 1).sum(), (y_train == 0).sum()

(32484, 32484)

In [61]:
(y_train == 0).sum()/len(y_train)

0.5

SMOTE

In [62]:
param_dist = {
    'n_estimators': randint(50, 200),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 0.5)
}

# Step 5: Initialize XGBoost Model
xgb_model = XGBClassifier(random_state=101, eval_metric='logloss')

# Step 6: Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings that are sampled
    cv=5,       # 5-fold cross-validation
    verbose=2,
    random_state=101,
    n_jobs=-1   # Use all available cores
)

# Step 7: Fit the model
random_search.fit(x_train, y_train)

# Step 8: Evaluate the best model
best_xgb_model = random_search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [63]:
# best_xgb_model.predict(x_resampled)

In [64]:
# accuracy_score(y_true=y_resampled, y_pred=best_xgb_model.predict(x_resampled))

In [65]:
# accuracy_score(y_true=y_test, y_pred=best_xgb_model.predict(x_test))

In [66]:
y_pred=best_xgb_model.predict(x_test)

In [67]:
status_list = ['Approved' if x == 1 else 'Rejected' for x in y_pred]

test_output.drop(['Logistic Regression Output', 'Decision Tree Output', 'Random Forest Output', 'XGBoost Output'], axis=1)
test_output['XGBoost Output'] = status_list

with pd.ExcelWriter('Output.xlsx', mode='a', engine='openpyxl') as writer:
    test_output.to_excel(writer, sheet_name='Predictions on Entire Dataset with sensitive fields removed', index=False)

# test_output.to_excel(writer, sheet_name='Predictions on Entire Dataset with sensitive fields removed')
# writer.save()

print("Classification Report for test set:")
print(classification_report(y_true=y_test, y_pred=best_xgb_model.predict(x_test)))
print("\nClassification Report for train set:")
print(classification_report(y_true=y_train, y_pred=best_xgb_model.predict(x_train)))

C:\Users\unnat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Classification Report for test set:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       635
           1       1.00      1.00      1.00      3619

    accuracy                           1.00      4254
   macro avg       1.00      0.99      1.00      4254
weighted avg       1.00      1.00      1.00      4254


Classification Report for train set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     32484
           1       1.00      1.00      1.00     32484

    accuracy                           1.00     64968
   macro avg       1.00      1.00      1.00     64968
weighted avg       1.00      1.00      1.00     64968



In [68]:
# cm = confusion_matrix(y_test, y_pred)

# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot()

# plt.show()

In [69]:
# explainer = shap.Explainer(best_xgb_model, x_resampled)

# shap_res = explainer.shap_values(x_resampled)
# shap.summary_plot(shap_res, x_resampled)

In [70]:
# shap_res = explainer.shap_values(x_test)
# shap.summary_plot(shap_res, x_test)

In [71]:
# import random

In [72]:
# random_index = random.randint(0, len(x_test) - 1)
# sample = x_test.iloc[random_index]

# y_sample = y_test.iloc[random_index]

In [73]:
# random_index

In [74]:
# sample

In [75]:
# y_sample

In [76]:
# explainer = shap.Explainer(best_xgb_model, x_resampled)
# shap_values = explainer(x_resampled)

In [77]:
# shap_values

In [78]:
# shap_res_test = explainer.shap_values(x_test)
# shap.summary_plot(shap_res_test, x_test)

In [79]:
# shap.plots.beeswarm(shap_values, max_display=30, color=plt.get_cmap("cool"))

In [80]:
# shap.plots.waterfall(shap_values[0], max_display=30)